In [1]:
from medtracker import *
from medtracker import database

/Users/ryanneff/suretify/medtracker/medtracker/__init__.py:14: ExtDeprecationWarning: Importing flask.ext.mail is deprecated, use flask_mail instead.
  from flask.ext.mail import Mail
/Users/ryanneff/suretify/medtracker/medtracker/__init__.py:15: ExtDeprecationWarning: Importing flask.ext.login is deprecated, use flask_login instead.
  import flask.ext.login as flask_login


In [2]:
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.dates as mdates

import pandas as pd
import numpy as np

import datetime
import plotly.graph_objects as go

from scipy.optimize import curve_fit

import warnings
warnings.filterwarnings('ignore')

In [149]:
patients = models.Patient.query.all()
devices = models.Device.query.all()

In [150]:
print(len(patients))
print(len(devices))

4
7


In [151]:
sres = []
for p in patients:
    sr = [s.to_dict() for s in p.surveys.all()]
    sres.extend(sr)

sres = pd.DataFrame(sres)

In [6]:
def number_complete(patients,last_days):
    start_date = (datetime.datetime.utcnow() - datetime.timedelta(days=last_days-1)+datetime.timedelta(days=1)).date()
    outcount = 0
    for p in patients:
        if len(p.surveys.filter(models.SurveyResponse.start_time < start_date).all())>0:
            outcount += 1
    return outcount

In [ ]:
df = pd.DataFrame(sres.groupby([sres.end_time.dt.floor('d')])["end_time"].count())
df

In [152]:
def plotlyBarplot(x=None,y=None,hue=None,data=None,ylabel="",xlabel="",title="",
                    width=600,height=400,colors=["rgba"+str(i) for i in cm.get_cmap("Set1").colors],
                    stacked=False,percent=False,ordered=False,xtype="category",
                  grouped=False,order2=True,show_legend=False):
    yaxis=go.layout.YAxis(
            title=ylabel,
            automargin=True,
            titlefont=dict(size=12),
        )

    xaxis=go.layout.XAxis(
            title=xlabel,
            automargin=True,
            titlefont=dict(size=12),
            type=xtype
        )

    layout = go.Layout(
        autosize=True,
        width=width,
        height=height,
        yaxis=yaxis,
        xaxis=xaxis
    )
    titledict= {'text': title,
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'}

    fig = go.Figure(layout=layout)
    fig.update_layout(xaxis_type = xtype)
    ylab = y
    xlab = x
    
    ix = 0
    catorder = list(data.groupby(xlab).sum().index)
    if hue != None:
        #data = data[[type(i)!=float for i in data[hue]]]
        if percent:
            totalcounts = data.groupby(xlab).sum()[ylab]
        for hue,group in data.groupby(hue):
            counts = group.groupby(xlab).sum()[ylab]
            if percent:
                counts = round(counts/totalcounts.loc[counts.index,]*100,1)
            x = [str(i) for i in counts.index]
            y = counts.values
            trace = go.Bar(x=x,y=y,
                           text=y,
                           textposition='auto',
                           orientation='v',
                         marker=dict(color=colors[ix]),
                         name="%s"%(hue))
            ix += 1
            fig.add_trace(trace)
    else:
        if grouped:
            counts = data.groupby(xlab).sum()[ylab]
            x = [str(i) for i in counts.index]
        else:
            counts = data[ylab]
            x = [str(i) for i in data[xlab]]
        y = counts.values
        trace = go.Bar(x=x,y=y,
                       text=y,
                       textposition='auto',
                       orientation='v',
                     marker=dict(color=colors[ix]),
                     name="%s"%(ylabel))
        ix += 1
        fig.add_trace(trace)
    fig.update_layout(title=titledict)
    if ordered:
        if stacked:
            if percent==False:
                fig.update_layout(barmode='stack',xaxis={'categoryorder':'total ascending'})
            else:
                fig.update_layout(barmode='stack',xaxis={'categoryorder':'category ascending'})
        else:
            fig.update_layout(xaxis={'categoryorder':'total ascending'})
    else:
        if stacked:
            fig.update_layout(barmode='stack')
    if order2:
        fig.update_layout(xaxis={"categoryorder":"array","categoryarray":catorder})
    fig.update_layout(showlegend=show_legend)
    return fig

In [153]:
sres = []
for p in patients:
    sr = [s.to_dict() for s in p.surveys.all()]
    sres.extend(sr)

sres = pd.DataFrame(sres)
sres["date"] = sres.end_time.dt.floor('d')
sres = sres.groupby(["date","uniq_id"]).first()
sres = sres.reset_index()

In [156]:
from plotly import offline
import pandas as pd
import numpy as np
import datetime
import plotly.graph_objects as go

patients = models.Patient.query.all()
devices = models.Device.query.all()

def model_to_pd(model):
    res = [r.to_dict() for r in model.query.all()]
    return pd.DataFrame(res)

sres = []
for p in patients:
    sr = [s.to_dict() for s in p.surveys.all()]
    sres.extend(sr)

sres = pd.DataFrame(sres)
sres["date"] = sres.end_time.dt.floor('d')
sres = sres.groupby(["date","uniq_id"]).first()
sres = sres.reset_index()

devs = pts = model_to_pd(models.Device)
devs_per_day = pd.DataFrame(devs.groupby([devs.creation_time.dt.floor("d")])["creation_time"].count())
devs_per_day.columns = ["daily_new_devices"]
pts = model_to_pd(models.Patient)
pts_per_day = pd.DataFrame(pts.groupby([pts.creation_time.dt.floor("d")])["creation_time"].count())
pts_per_day.columns = ["daily_registered_students"]
res_per_day = pd.DataFrame(sres.groupby([sres.end_time.dt.floor('d')])["end_time"].count())
res_per_day.columns = ["daily_total_surveys"]
comp_per_day = pd.DataFrame(sres[sres.completed==True].groupby([sres.end_time.dt.floor('d')])["end_time"].count())
comp_per_day.columns = ["daily_completed_surveys"]
exit_per_day = pd.DataFrame(sres[sres.completed==False].groupby([sres.end_time.dt.floor('d')])["end_time"].count())
exit_per_day.columns = ["daily_exited_surveys"]
df = pd.merge(pts_per_day,res_per_day,left_index=True,right_index=True,how="outer")
df = pd.merge(df,comp_per_day,left_index=True,right_index=True,how="outer")
df = pd.merge(df,exit_per_day,left_index=True,right_index=True,how="outer")
df = pd.merge(df,devs_per_day,left_index=True,right_index=True,how="outer")
begin_time = datetime.datetime.utcnow().date() - datetime.timedelta(days=14)
df = df.reindex(pd.date_range(begin_time, datetime.datetime.utcnow().date())).fillna(0).astype(int)
df["total_registered_students"] = df["daily_registered_students"].cumsum()
df["total_completed_surveys"] = df["daily_completed_surveys"].cumsum()
df["total_devices"] = df["daily_new_devices"].cumsum()
df.reset_index(inplace=True)
df = df.sort_values(by="index",ascending=True)
df["index"] = [datetime.datetime.strftime(a,"%D") for a in df["index"]]
#plotlyBarplot(data=df,x="index",y="daily_new_devices",width=400,title="New Devices")
fig1 = plotlyBarplot(data=df,x="index",y="total_devices",width=500,height=300,title="Total Devices")
fig2 = plotlyBarplot(data=df,x="index",y="total_registered_students",width=500,height=300,title="Total Registered Students")
#plotlyBarplot(data=df,x="index",y="daily_completed_surveys",width=400,title="Daily Completed Screenings")
df["daily_uncompleted_surveys"] = df["total_registered_students"] - df["daily_total_surveys"]
df2 = df.loc[:,["index","daily_uncompleted_surveys","daily_completed_surveys","daily_exited_surveys"]]
df2.columns = ["index","Not Completed","Completed","Exited"]
df2 = df2.melt(id_vars="index")
fig3 = plotlyBarplot(data=df2,x="index",y="value",hue="variable",width=600,height=300, title="Screening Status",
                     stacked=True,colors=["green","red","orange"])
fig3.show()

In [157]:
sres

,date,uniq_id,completed,end_time,exited,id,message,session_id,start_time,survey_id,user_id
0,2020-06-14,1,True,2020-06-14 02:33:53.821905,False,13,You can attend class or work.,GX42S2IGM3A6BY2GKUA7GINIGSDNKBG6,2020-06-14 02:33:23.182054,1,None
1,2020-06-15,1,False,2020-06-15 15:07:53.608409,True,24,"You should stay home, based on your reported s...",VCUMYBBB35SF9D3NJI322DDU2RPW4F79,2020-06-15 15:07:46.693521,1,None
2,2020-06-18,5,True,2020-06-18 19:02:30.197768,False,27,You can attend class or work.,5L4QLDZK7LXULGMICDC3YY9BVTHRAGHU,2020-06-18 19:02:25.913253,1,None


In [82]:
pts_sres = pd.merge(pts,sres,left_on="id",right_on="uniq_id",how="inner")

In [83]:
pts.set_index("creation_time")

,age,email,fullname,id,location,mrn,phone,program,user_id,year
creation_time,,,,,,,,,,
2020-06-14 02:22:33,28,raneff@gmail.com,Ryan Ashford Neff,1,Other housing (other locations),XQ8Z858FI662UJTV,19739608144,Dual degree programs (e.g. MD/PhD),1,2023
2020-06-17 14:38:22,,,,2,Student housing (e.g. Aron Hall),XTM47QRAJZFA3MY6,,MD,1,2021
2020-06-18 17:33:33,,,,3,Student housing (e.g. Aron Hall),9AHAQ5LDFIJW6E94,,MD,1,2020
2020-06-18 17:36:55,,,,4,Student housing (e.g. Aron Hall),75HZS2G4PCPGEIRJ,,MD,1,2020


In [194]:
pts["location"] = [str(i) for i in pts["location"]]
pts["program"] = [str(i) for i in pts["program"]]
reg_per_year = plotlyBarplot(data=pd.DataFrame(pts.groupby(["program","location"]).count()["id"]).reset_index(),y="id",x="location")

In [199]:
plotlyBarplot(data=pd.DataFrame(pts.groupby(["program","location"]).count()["id"]).reset_index(),y="id",x="location",hue="program",
             stacked=True)

In [91]:
sres

,date,uniq_id,completed,end_time,exited,id,message,session_id,start_time,survey_id,user_id
0,2020-06-14,1,True,2020-06-14 02:33:53.821905,False,13,You can attend class or work.,GX42S2IGM3A6BY2GKUA7GINIGSDNKBG6,2020-06-14 02:33:23.182054,1,None
1,2020-06-15,1,False,2020-06-15 15:07:53.608409,True,24,"You should stay home, based on your reported s...",VCUMYBBB35SF9D3NJI322DDU2RPW4F79,2020-06-15 15:07:46.693521,1,None


In [111]:
i = pd.date_range(begin_time, datetime.datetime.utcnow().date())[10]
sres[(sres.end_time<(i+datetime.timedelta(days=1)))&(sres.end_time>i)]

,date,uniq_id,completed,end_time,exited,id,message,session_id,start_time,survey_id,user_id
0,2020-06-14,1,True,2020-06-14 02:33:53.821905,False,13,You can attend class or work.,GX42S2IGM3A6BY2GKUA7GINIGSDNKBG6,2020-06-14 02:33:23.182054,1,None


In [97]:
i = pd.date_range(begin_time, datetime.datetime.utcnow().date())[10]

Timestamp('2020-06-14 00:00:00', freq='D')

In [183]:
from collections import defaultdict
pts_df = pts.set_index("creation_time")
years = list(range(2020,2024))
reg_per_year = defaultdict(list)
response_hits = defaultdict(list)
outdf = []
begin_time = datetime.datetime.utcnow().date() - datetime.timedelta(days=6)
for i in pd.date_range(begin_time, datetime.datetime.utcnow().date()):
    for y in years:
        pts_yr = pts_df[pts_df.year==y]
        hits = pts_yr[(pts_yr.index<i+datetime.timedelta(days=1))&(pts_yr.index>i)]
        response_hits = set(sres[(sres.end_time<(i+datetime.timedelta(days=1)))&(sres.end_time>i)].uniq_id)
        complete_hits = set(sres[(sres.end_time<(i+datetime.timedelta(days=1)))&(sres.end_time>i)&(sres.completed==True)].uniq_id)
        exit_hits = set(sres[(sres.end_time<(i+datetime.timedelta(days=1)))&(sres.end_time>i)&(sres.exited==True)].uniq_id)
        reg_per_year[y].extend(hits["id"])
        outdf.append([i,y,len(reg_per_year[y]),
                      len(response_hits.intersection(set(reg_per_year[y]))),
                     len(complete_hits.intersection(set(reg_per_year[y]))),
                     len(exit_hits.intersection(set(reg_per_year[y]))),
                     len(set(reg_per_year[y]).difference(response_hits))])
outdf = pd.DataFrame(outdf,columns=["date","year","total_registered","total_responded","Completed","Exited","Not Completed"])

In [189]:
todaydf = outdf[outdf["date"]==datetime.datetime.utcnow().date()].loc[:,["year","Completed","Exited","Not Completed"]].melt(id_vars="year")
plotlyBarplot(data=todaydf,x="year",y="value",hue="variable",stacked=True,ylabel="# Students",xlabel="Program Year",
             title="Compliance Today",colors=["green","red","orange"])

In [45]:
output = []
for i in pd.date_range(begin_time, datetime.datetime.utcnow().date()):
    print(i)

2020-06-04 00:00:00
2020-06-05 00:00:00
2020-06-06 00:00:00
2020-06-07 00:00:00
2020-06-08 00:00:00
2020-06-09 00:00:00
2020-06-10 00:00:00
2020-06-11 00:00:00
2020-06-12 00:00:00
2020-06-13 00:00:00
2020-06-14 00:00:00
2020-06-15 00:00:00
2020-06-16 00:00:00
2020-06-17 00:00:00
2020-06-18 00:00:00


In [40]:
pts_per_day.reset_index()

,year,creation_time,daily_registered_students
0,2020,2020-06-18,2
1,2021,2020-06-17,1
2,2023,2020-06-14,1


In [41]:
res_per_day

daily_completed_surveys
year end_time                           
2023 2020-06-14                        1
     2020-06-15                        1

In [30]:
pts = model_to_pd(models.Patient)
pts_per_day = pd.DataFrame(pts.groupby([pts.year,pts.creation_time.dt.floor("d")])["creation_time"].count())
pts_per_day.columns = ["daily_registered_students"]

In [36]:
pts_per_day

,,daily_registered_students
year,creation_time,
2020,2020-06-18,2
2021,2020-06-17,1
2023,2020-06-14,1


In [31]:
res_per_day = pd.DataFrame(pts_sres.groupby([pts_sres.year,pts_sres.end_time.dt.floor('d')])["end_time"].count())
res_per_day.columns = ["daily_completed_surveys"]
df = pd.merge(pts_per_day,res_per_day,left_index=True,right_index=True,how="outer")

NotImplementedError: merging with both multi-indexes is not implemented

In [37]:
res_per_day

daily_completed_surveys
year end_time                           
2023 2020-06-14                        1
     2020-06-15                        1

In [ ]:
%run ~/zhang-ipy/Programs/ipy_setup.py
%matplotlib inline

In [ ]:
(datetime.datetime.now()-datetime.timedelta(days=1)).date()

In [ ]:
survey = models.Survey.query.get(1)
sr = survey.responses.filter(models.SurveyResponse.start_time > (datetime.datetime.now()-datetime.timedelta(days=6)).date()).all()
responses = []
for sre in sr: responses.extend([r.to_dict() for r in sre.responses])
qres = responses

qres = pd.DataFrame(qres)
qres["date"] = qres.time.dt.floor('d')
qres = qres.groupby(["date","question_id","uniq_id"]).first()
qres = qres.reset_index()

In [ ]:
qres

In [ ]:
g["count"] = 1

In [ ]:
list(pltdf.groupby("date").count().index)

In [8]:
e = [str(a.date()) for a in list(pd.date_range(datetime.datetime.now().date()-datetime.timedelta(days=7),datetime.datetime.now().date()))]

In [ ]:
pltdf.groupby("date").sum()["count"]

In [20]:
refmt = []
for ix,row in pltdf.iterrows():
    row = list(row)
    for rr in row[1].split(";"):
        refmt.append([row[0],rr,row[2]])
pltdf = pd.DataFrame(refmt,columns=pltdf.columns)

,date,response,count
0,2020-06-04,None of the above,0.0
1,2020-06-05,None of the above,0.0
2,2020-06-06,None of the above,0.0
3,2020-06-07,None of the above,0.0
4,2020-06-08,None of the above,1.0
5,2020-06-09,None of the above,1.0
6,2020-06-09,None of the above,1.0
7,2020-06-10,None of the above,1.0
8,2020-06-11,Temperature >100F,1.0
9,2020-06-11,Cough,1.0


In [21]:
survey_id = 1
sr = models.Survey.query.get(survey_id).responses
responses = []
figs = []
for sre in sr: responses.extend([r.to_dict() for r in sre.responses])

qres = pd.DataFrame(responses)
qres["date"] = qres.time.dt.floor('d')
qres = qres.groupby(["date","question_id","uniq_id"]).first()
qres = qres.reset_index()

for n,g in qres.groupby("question_id"):
    title = list(g.question_title)[0]
    choices = list(g.question_choices)[0]
    choices = ast.literal_eval(choices) if choices != "" else {}
    kind = list(g.question_type)[0]
    xtype = "category"
    pltdf = g.loc[:,["date","response"]]
    pltdf["count"] = 1
    pltdf.sort_values("date",inplace=True)
    pltdf["date"] = pltdf["date"].astype(str)
    refmt = []
    for ix,row in pltdf.iterrows():
        row = list(row)
        for rr in row[1].split(";"):
            refmt.append([row[0],rr,row[2]])
    pltdf = pd.DataFrame(refmt,columns=pltdf.columns)
    pltdf.set_index("date",inplace=True)
    for ix in e:
        if ix not in pltdf.index:
            pltdf.loc[ix] = [None,0]
    pltdf.sort_index(inplace=True)
    pltdf.reset_index(inplace=True)
    pltdf=pltdf.fillna(method="bfill")
    fig = plotlyBarplot(data=pltdf,x="date",y="count",hue="response",
                        xtype=xtype,grouped=True,ordered=False,stacked=True,order2=True,show_legend=True)
    figs.append(fig)

In [22]:
for fig in figs:
    fig.show()

In [ ]:
plotlyBarplot(data=qres[qres["question_id"]==1],x="question_title",y="response")

In [ ]:
from plotly import offline

In [ ]:
df.loc[:,["index","total_registered_students","daily_completed_surveys"]].melt(id_vars="index")

In [ ]:
print(len(patients))
print(number_complete(patients,1))

In [ ]:
def destroy_responses():
    for a in models.SurveyResponse.query.all():
        db.session.delete(a)
    db.session.commit()
    for a in models.QuestionResponse.query.all():
        db.session.delete(a)
    db.session.commit()
    for a in models.Patient.query.all():
        db.session.delete(a)
    db.session.commit()
    for a in models.Device.query.all():
        db.session.delete(a)
    db.session.commit()

In [120]:
["rgba"+str(i) for i in cm.get_cmap("Paired").colors]

['rgba(0.6509803921568628, 0.807843137254902, 0.8901960784313725)',
 'rgba(0.12156862745098039, 0.47058823529411764, 0.7058823529411765)',
 'rgba(0.6980392156862745, 0.8745098039215686, 0.5411764705882353)',
 'rgba(0.2, 0.6274509803921569, 0.17254901960784313)',
 'rgba(0.984313725490196, 0.6039215686274509, 0.6)',
 'rgba(0.8901960784313725, 0.10196078431372549, 0.10980392156862745)',
 'rgba(0.9921568627450981, 0.7490196078431373, 0.43529411764705883)',
 'rgba(1.0, 0.4980392156862745, 0.0)',
 'rgba(0.792156862745098, 0.6980392156862745, 0.8392156862745098)',
 'rgba(0.41568627450980394, 0.23921568627450981, 0.6039215686274509)',
 'rgba(1.0, 1.0, 0.6)',
 'rgba(0.6941176470588235, 0.34901960784313724, 0.1568627450980392)']

In [121]:
def plotlyCountplot(x=None,y=None,hue=None,data=None,ylabel="",xlabel="",title="",
                    width=600,height=400,colors=["rgba"+str(i) for i in cm.get_cmap("Paired").colors],
                    stacked=False,percent=False):
    xaxis=go.layout.XAxis(
            title=ylabel,
            automargin=True,
            titlefont=dict(size=12),
        )

    yaxis=go.layout.YAxis(
            title=xlabel,
            automargin=True,
            titlefont=dict(size=12),
            type='category'
        )

    layout = go.Layout(
        autosize=True,
        width=width,
        height=height,
        yaxis=yaxis,
        xaxis=xaxis
    )
    titledict= {'text': title,
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'}

    fig = go.Figure(layout=layout)
    ylab = y
    xlab = x

    ix = 0
    if hue != None:
        data = data[[type(i)!=float for i in data[hue]]]
        if percent:
            data["count"] = 1
            totalcounts = data.groupby(ylab).count()["count"]
        for hue,group in data.groupby(hue):
            group["count"] = 1
            counts = group.groupby(ylab).count()["count"]
            if percent:
                counts = round(counts/totalcounts.loc[counts.index,]*100)
            x = [str(i) for i in counts.index]
            y = counts.values
            trace = go.Bar(x=y,y=x,
                           text=y,
                           textposition='auto',
                           orientation='h',
                         marker=dict(color=colors[ix]),
                         name="%s"%(hue))
            ix += 1
            fig.add_trace(trace)
    else:
        data["count"] = 1
        counts = data.groupby(ylab).count()["count"]
        x = [str(i) for i in counts.index]
        y = counts.values
        trace = go.Bar(x=y,y=x,
                       text=y,
                       textposition='auto',
                       orientation='h',
                     marker=dict(color=colors[ix]),
                     name="%s"%(ylabel))
        ix += 1
        fig.add_trace(trace)
    fig.update_layout(title=titledict)
    if stacked:
        if percent==False:
            fig.update_layout(barmode='stack',yaxis={'categoryorder':'total ascending'})
        else:
            fig.update_layout(barmode='stack',yaxis={'categoryorder':'category descending'})
    else:
        fig.update_layout(yaxis={'categoryorder':'total ascending'})
    return fig
